Notebook elaborado para estudar as características relacionadas à assinatura magnetométrica de corpos de diferentes geometrias.
Criado para a disciplina Geofísica II da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Esta demonstração utiliza a modelagem de dados gravimétricos implementados no pacote *open source* [SimPEG v.14.3](https://github.com/simpeg/simpeg) (COCKETT et al., 2015), distribuído sob os termos da [licença MIT](https://github.com/simpeg/simpeg/blob/master/LICENSE). Não foram feitas modificações nos códigos utilizados.


Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
emerson.almeida@ufu.br<br>

<p>&nbsp;</p>
Última modificação: 23/09/2022

---

**Referência:**

COCKETT, R.; KANG, S.; HEAGY, L. J.; PIDLISECKY, A.; OLDENBURG, D. W. SimPEG: An open source framework for simulation and gradient based parameter estimation in geophysical applications. **Computers & Geosciences**, v.85, Part A, p. 142-154, 2015.

# Instruções iniciais

Neste notebook você irá visualizar a anomalia magnética de campo total gerada por corpos de geometria simples. Para isto leia atentamente as instruções apresentadas antes de cada célula e execute-as uma a uma, para acompanhar o processo corretamente.

Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

<p>&nbsp;</p>

### Passo 01 - Preparação

Primeiramente vamos preparar o ambiente para a execução da demonstração. Será preciso utilizar o [SimPEG](https://simpeg.xyz/), que é um pacote open source para modelagem de dados geofísicos. Se você estiver executando este notebook pelo Binder ou pelo Colab execute a célula abaixo para instalar o SimPEG remotamente.

Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado. 

**Importante!** Nem todas as células irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [   ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada.

In [ ]:
import modulos.potenciais as mp

### Passo 02 - Configuração do campo geomagnético

A assinatura de uma anomalia magnética está sujeita às características de inclinação, declinação e intensidade do campo geomagnético na região onde a aquisição é feita, de forma que a anomalia observada sobre um mesmo corpo pode apresentar comportamentos diferentes em diferentes locais do mundo. Iremos definir inicialmente as características do campo geomagnético no Polo Norte magnético, onde a inclinação do campo é igual a 90.0º e a declinação é igual a 0.0º. Vamos utilizar uma intensidade teórica de 50000.0 nT.

Execute a célula abaixo para configurar estas características.

In [ ]:
intensidade = 50000.0      # intensidade total em nT
inclinacao = 90.0          # inclinação em graus
declinacao = 00.0          # declinação em graus

### Passo 03 - Anomalia causada por corpos de geometria simples

Primeiramente será configurado um corpo de forma esférica a uma profundidade de 50.0 metros, com raio igual a 150.0 m. Execute a célula abaixo para configurar este corpo.

In [ ]:
meu_modelo = mp.esfera(r=150.0, z=-50.0, chi=1e-2)
#meu_modelo = mp.cubo(l=300.0, z=-50.0, chi=1e-2)
#meu_modelo = mp.dique(l=50.0, z=-50.0, chi=1e-2)
#meu_modelo = mp.sill(l=800.0, h=50.0, z=-50.0, chi=1e-2)

Execute a célula abaixo para exibir vistas em corte do modelo nas três direções.

In [ ]:
meu_modelo.plota_corte('Y', 0.0)
meu_modelo.plota_corte('X', 0.0)
meu_modelo.plota_corte('Z', 75.0)

Uma vez verificada a geometria do modelo com o corpo desejado, execute a célula abaixo para executar a simulação da aquisição em uma malha de pontos localizada na superfície do modelo (i.e., à elevação de 0.0 m).

*Esta etapa pode demorar alguns minutos.*

In [ ]:
minha_aquisicao = mp.aquisicao(meu_modelo, f=intensidade, inc=inclinacao, dec=declinacao)
%time minha_aquisicao.fwd(meu_modelo)

Por fim, execute a célula abaixo para obter o mapa de intensidade magnética total referente ao modelo configurado. Os valores numéricos entre os colchetes correspondem às posições dos perfis indicados no mapa (linhas tracejadas brancas) e mostradas à direita na figura. Você pode modificá-los, caso deseje.

In [ ]:
minha_aquisicao.plota_mapa([0.0, 0.0])

### Passo 04 - Alterando a geometria do corpo

Agora você poderá configurar corpos de outras formas geométricas e verificar a assinatura magnética característica de cada uma delas. Para isto, execute os seguintes passos na primeira célula do Passo 03:

**1-** Insira um símbolo **#** no início da linha que não aparece em itálico. Ao fazer isso, a fonte do texto aparecerá em itálico. Isto indica que esta linha está configurada como um comentário e não como uma linha de código e, portanto, não será interpretada pela linguagem como um comando a ser executado;

**2-** Escolha uma das linhas nesta célula que contém o tipo de estrutura que você gostaria de modelar (esfera, cubo, dique ou sill) e apague o símbolo **#** presente no início da linha em que há a estrutura desejada. Ao fazer isso esta linha deixará de ser um comentário e passará a ser interpretada como um comando a ser executado.

*Atenção*: Deixe apenas uma das linhas sem o símbolo **#**, pois caso contrário haverá sobreposição entre diferentes estruturas.

**3-** Opcionalmente você pode configurar diferentes dimensões e profundidades para os corpos a serem modelados, bem como diferentes valores de susceptibilidade magnética. Isto pode ser feito alterando-se os valores numéricos presentes na linha da estrutura correspondente de acordo com a seguinte relação:

$r$ = raio da esfera (metros);

$z$ = profundidade do topo do corpo (metros) - deve ser um valor < 0!;

$l$ = extensão lateral do cubo, do dique ou do sill (metros);

$h$ = espessura do sill (metros).

$chi$ = susceptibilidade magnética (SI)

### Passo 05 - Alterando as características do campo geomagnético

Agora você poderá configurar corpos de outras formas geométricas e verificar como a assinatura da anomalia se comporta em função da variação da intensidade, inclinação e declinação do campo magnétio. Para isto, execute o seguinte procedimento na primeira célula do Passo 02:

**1-** Configure a intensidade do campo geomagnético para a intensidade = 50000.0 nT, inclinação = 0.0º e declinação = 0.0º. Estas compoentes correspondem a um campo teórico no equador magnético. Verifique como o comportamento da anomalia se altera.

**2-** Configure as características do campo geomagnético para aquelas correspondentes às coordenadas onde encontra-se o campus da Universidade Federal de Uberlândia em Monte Carmelo (latitude = -18.723889°, longitude = -47.523889°, altitude = 890.0 m). Utilize os valores de intensidade = 23170.0 nT, inclinação = -32.9445º e declinação = -21.8288º calculados pelo IGRF-13 no dia 30/03/2021, data da modificação mais recente deste notebook. Verifique como o comportamento da anomalia se altera.

**3-** Configure as características do campo geomagnético para outras localidades do globo terrestre. Estas características podem ser obtidas a partir da ferramenta de cálculo do IGRF disponível no [site da NOAA](https://www.ngdc.noaa.gov/geomag/calculators/magcalc.shtml#igrfwmm). Verifique como as características da anomalia gerada se alteram apenas em função da localização deste corpo no globo terrestre, mesmo que a sua geometria não seja modificada.

**4-** Configure as características do campo geomagnético para uma mesma localidade (pode ser a localização do campus da UFU em Monte Carmelo dada no item 2), porém em diferentes épocas. Estas informações também podem ser obtidas a partir da ferramenta de cálculo do IGRF disponível no [site da NOAA](https://www.ngdc.noaa.gov/geomag/calculators/magcalc.shtml#igrfwmm). Verifique como as características da anomalia gerada por um mesmo corpo em uma localidade específica se modifica em função da variação secular do campo geomagnético. Sugestão: Começando pelo ano de 2021, considere datas retrocedendo no tempo em períodos de 5 anos.